# Alignment
## Load libraries and data

In [2]:
import pandas as pd
from collections import Counter

from alignment import align_on_column
from preprocessing import substitute_col_by_dict

In [3]:
bottins = pd.read_pickle("data/bottins_prep.pkl")
streets = pd.read_pickle("data/streets_prep.pkl")
unique_short_s = pd.read_pickle("data/unique_short_streets.pkl")
non_unique_short_s = pd.read_pickle("data/not_unique_short_streets.pkl")

In [4]:
streets.head(3)

,id1791,nom_voie,unprocessed_voie,voie_long,voie_short,id2022,L_VOIE,L_COURTMIN,L_LONGMIN,street_short,street_abbr,street_long,source,streetname,streetname_short
0,1162.0,deux Boules,rue des deux Boules,rue des deux boules,deux boules,750005865.0,DEUX BOULES,R. des Deux Boules,Rue des Deux Boules,deux boules,r. des deux boules,rue des deux boules,both,rue des deux boules,deux boules
1,1531.0,Jean Lantier,rue Jean Lantier,rue jean lantier,jean lantier,750005921.0,JEAN LANTIER,R. Jean Lantier,Rue Jean Lantier,jean lantier,r. jean lantier,rue jean lantier,both,rue jean lantier,jean lantier
2,3.0,Orfèvres,rue des Orfèvres,rue des orfevres,orfevres,750006633.0,ORFEVRES,R. des Orfèvres,Rue des Orfèvres,orfevres,r. des orfevres,rue des orfevres,both,rue des orfevres,orfevres


## Align data

In [5]:
long_aligned, not_aligned = align_on_column(df_not_aligned = bottins, df_streets = streets, 
                    mergeOnLeft="street", mergeOnRight="streetname", align_method="perfect")
u_short_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= unique_short_s, 
                    mergeOnLeft="street", mergeOnRight="streetname_short", align_method="perfect short")
nu_short_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= non_unique_short_s, 
                    mergeOnLeft="street", mergeOnRight="streetname_short", align_method="perfect short")                

c:\Users\Lea\Documents\Uni Aachen\4. Semester\DH-405 Foundation Digital Humanities\project\Paris_City_Project\alignment.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newly_aligned["align_method"] = align_method


Joining on street and streetname, method:perfect
#total aligned: 1287622, newly aligned: 1287622, not aligned: 3198573
Joining on street and streetname_short, method:perfect short
#total aligned: 1177527, newly aligned: 1177527, not aligned: 2021046
Joining on street and streetname_short, method:perfect short
#total aligned: 938953, newly aligned: 938953, not aligned: 1082093


## Substitute words by dictionary

In [6]:
word_dict = {"boulevard": "boulevard de",
                "boulevard de de ": "boulevard de ",
                "boulevard de d'": "boulevard d'",
                "boulevards": "boulevard des",
                "damede": "dame de",
                "damedes": "dame des",
                "faubourgsaint": "faubourg saint",
                "faubourgpoissonniere": "faubourg poissonniere",
                "faubourgdu": "faubourg du",
                "faubourgmontmartre": "faubourg montmartre",
                "quai jemmapes": "quai de jemmapes",
                "boulevards italiens": "boulevard des italiens",
                "villeneuve": "ville neuve",
                "quai valmy": "quai de valmy",
                "avenue wagram": "avenue de wagram",
                "boulevard de montparnasse": "boulevard du montparnasse"
                }

# substitute abbreviations
not_aligned["street"] = substitute_col_by_dict(not_aligned["street"], word_dict)

In [7]:
long_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= streets, 
                    df_aligned = long_aligned, mergeOnLeft="street", mergeOnRight="streetname", 
                    align_method="perfect")
u_short_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= unique_short_s, 
                    df_aligned= u_short_aligned, mergeOnLeft="street", mergeOnRight="streetname_short", 
                    align_method="perfect short")
nu_short_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= non_unique_short_s, 
                    df_aligned= nu_short_aligned, mergeOnLeft="street", mergeOnRight="streetname_short", 
                    align_method="perfect short")                

Joining on street and streetname, method:perfect
#total aligned: 1348632, newly aligned: 61010, not aligned: 1021083
Joining on street and streetname_short, method:perfect short
#total aligned: 1187277, newly aligned: 9750, not aligned: 1011333
Joining on street and streetname_short, method:perfect short
#total aligned: 939959, newly aligned: 1006, not aligned: 1010327


In [8]:
most_common = Counter(not_aligned["street"]).most_common(100)
for t in most_common:
    rue, occur = t
    last = rue.split()[-1]
    print(rue, occur, [x for x in streets["streetname"] if last in x], "\n")

notre dame de  nazareth 9463 ['rue notre dame de nazareth', 'rue de nazareth'] 

grenelle saint germain 9182 ["rue saint germain l'auxerrois", 'rue des fosses saint germain des pres', 'foire saint germain', 'preau de la foire saint germain', "rue des fosses saint germain l'auxerrois", "cloitre saint germain l'auxerrois", 'route de saint germain a paris', "rue des pretres saint germain l'auxerrois", 'rue germaine tailleferre', 'rue du docteur germain see', 'cite germain pilon', 'rue germain pilon', 'rue germaine richier', 'rue germaine krull', 'rue sophie germain', 'place saint germain des pres', 'rue germaine de stael', 'boulevard saint germain', 'rue germaine poinso chapuis'] 

allemagne 6737 [] 

rochechouart 5863 ['rue marguerite de rochechouart', 'boulevard marguerite de rochechouart'] 

avenue d'orleans 5696 ["rue d'orleans", "rue d'orleans", "quay d'orleans", "rue neuve d'orleans", "rue neuve d'orleans", "route d'orleans", "square d'orleans", "les portiques d'orleans", "quai d'or

In [9]:
# to check if something exists in the streets dataset
print([x for x in streets["streetname"] if "boulevards" in x])
print(streets[streets["streetname_short"]=="la fayette"])

[]
      id1791 nom_voie unprocessed_voie voie_long voie_short       id2022  \
2621     NaN      NaN              NaN       NaN        NaN  750004617.0   

          L_VOIE     L_COURTMIN       L_LONGMIN street_short    street_abbr  \
2621  LA FAYETTE  R. la Fayette  Rue la Fayette   la fayette  r. la fayette   

         street_long source      streetname streetname_short  
2621  rue la fayette   2022  rue la fayette       la fayette  


In [10]:
# get most common in not aligned data
Counter([x for x in not_aligned["street"] if "." in x]).most_common(100)

[('faub.poissonniere', 1394),
 ('grenelle saint germ.', 1014),
 ('faub.montmartre', 765),
 ('four saint germ.', 404),
 ('j. j.rousseau', 371),
 ('fanb. saint martin', 362),
 ('fauh. saint martin', 345),
 ('fanb. saint antoine', 344),
 ('fanb. saint denis', 343),
 ('fauh. saint antoine', 303),
 ('f. saint martin', 302),
 ('fauh. saint denis', 296),
 ('saint dominique saint germ.', 285),
 ('neuve des pet. champs', 263),
 ('f. saint antoine', 258),
 ('saint maur popinc.', 244),
 ('croix des pet. champs', 239),
 ('grenelle saint g.', 223),
 ('f. saint denis', 222),
 ('n.d. de nazareth', 216),
 ('fanb. saint honore', 215),
 ('fanb. poissonniere', 213),
 ('sainte marguerite saint germ.', 209),
 ('ay. parmentier', 205),
 ('faub.. poissonniere', 204),
 ('fanb. du temple', 200),
 ("saint germ. l'auxerrois", 200),
 ('j.j. rousseau', 196),
 ('faub.. saint denis', 190),
 ('fauh. saint honore', 187),
 ('fauh. poissonniere', 183),
 ('faub.. saint martin', 179),
 ('boulevard de vol. taire', 179),
 ('

### Alignment without spaces

In [11]:
replace_spaces = {"\ ":"", "\|":"", "\.":"", "\:":"", "\'":""}
not_aligned["no_spaces"] = not_aligned["street"].replace(replace_spaces, regex=True)
streets["no_spaces_long"] = streets["streetname"].replace(replace_spaces, regex=True)
unique_short_s["no_spaces_short"] = unique_short_s["streetname_short"].replace(replace_spaces, regex=True)
non_unique_short_s["no_spaces_short"] = non_unique_short_s["streetname_short"].replace(replace_spaces, regex=True)

In [12]:
not_aligned.head(5)

,Unnamed: 0,page,row,Nom,Métier,Rue,Numéro,annee,street,no_spaces
0,bpt6k6282019m,144,0,Aaron,bronzes,passage Choiseal,72 et 74.,1855,passage choiseal,passagechoiseal
1,bpt6k6282019m,144,13,Abault et Coudray,charpentiers,Corbeau,23.,1855,corbeau,corbeau
2,bpt6k6282019m,144,14,Abault (Paul),libraire,quai des Angustins,9.,1855,quai des angustins,quaidesangustins
3,bpt6k6282019m,144,15,Abavid,vins,Beaujolais-da-Temple,7.,1855,beaujolais da temple,beaujolaisdatemple
4,bpt6k6282019m,144,16,Abazaer (Are),cristaux et porcelaines,Pei.Ecuries,26.,1855,pei.ecuries,peiecuries


In [13]:
long_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= streets, 
                    df_aligned = long_aligned, mergeOnLeft="no_spaces", mergeOnRight="no_spaces_long", 
                    align_method="no spaces perfect")
u_short_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= unique_short_s, 
                    df_aligned = u_short_aligned, mergeOnLeft="no_spaces", mergeOnRight="no_spaces_short",
                    align_method="no spaces perfect short")
nu_short_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= non_unique_short_s, 
                    df_aligned = nu_short_aligned, mergeOnLeft="no_spaces", mergeOnRight="no_spaces_short",
                    align_method="no spaces perfect short")

c:\Users\Lea\Documents\Uni Aachen\4. Semester\DH-405 Foundation Digital Humanities\project\Paris_City_Project\alignment.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newly_aligned["align_method"] = align_method


Joining on no_spaces and no_spaces_long, method:no spaces perfect
#total aligned: 1397378, newly aligned: 48746, not aligned: 963031
Joining on no_spaces and no_spaces_short, method:no spaces perfect short
#total aligned: 1248476, newly aligned: 61199, not aligned: 902077
Joining on no_spaces and no_spaces_short, method:no spaces perfect short
#total aligned: 962694, newly aligned: 22735, not aligned: 879353


In [14]:
nu_short_aligned.tail(10)

,Unnamed: 0,page,row,Nom,Métier,Rue,Numéro,annee,street,streetname_short,id1791,id2022,streetname,align_method,no_spaces,no_spaces_short
883637,bpt6k9780089g,788,97,Bossuat,couronnes funéraires,.Montgallet,23.,1922,.montgallet,montgallet,"[1172.0, nan]","[750003056.0, 750003055.0]","[rue montgallet, passage montgallet]",no spaces perfect short,montgallet,montgallet
884550,bpt6k9780089g,827,238,Caillat (Edmond),éténisterie,Faub. St-An. toine,24.,1922,faubourg saint an. toine,faubourg saint antoine,"[1170.0, nan]","[nan, 750003526.0]","[grande rue du faubourg saint antoine, rue du ...",no spaces perfect short,faubourgsaintantoine,faubourgsaintantoine
885280,bpt6k9780089g,858,79,Charpentier & Lemaitre,bar,Faub. St-An. toine,161.,1922,faubourg saint an. toine,faubourg saint antoine,"[1170.0, nan]","[nan, 750003526.0]","[grande rue du faubourg saint antoine, rue du ...",no spaces perfect short,faubourgsaintantoine,faubourgsaintantoine
888333,bpt6k9780089g,997,208,Blectro-Stock (L'),appareillage électrique,. Coquillière,31.,1922,. coquilliere,coquilliere,"[273.0, nan]","[750005879.0, 750006048.0]","[rue coquilliere, souterrain coquilliere]",no spaces perfect short,coquilliere,coquilliere
891653,bpt6k9780089g,1149,270,Jamaux (F.),art. de literie,Faub. St-An. toine,283.,1922,faubourg saint an. toine,faubourg saint antoine,"[1170.0, nan]","[nan, 750003526.0]","[grande rue du faubourg saint antoine, rue du ...",no spaces perfect short,faubourgsaintantoine,faubourgsaintantoine
891782,bpt6k9780089g,1156,228,Jollain (A.),margueterie,Faub. St-Antoi. ne,95.,1922,faubourg saint antoi. ne,faubourg saint antoine,"[1170.0, nan]","[nan, 750003526.0]","[grande rue du faubourg saint antoine, rue du ...",no spaces perfect short,faubourgsaintantoine,faubourgsaintantoine
892093,bpt6k9780089g,1169,277,Klémann (F.),ameublements,Faub. St-An. toine,62.,1922,faubourg saint an. toine,faubourg saint antoine,"[1170.0, nan]","[nan, 750003526.0]","[grande rue du faubourg saint antoine, rue du ...",no spaces perfect short,faubourgsaintantoine,faubourgsaintantoine
897399,bpt6k9780089g,1401,92,Pont (Mme),couturière,. Lamarck,145.,1922,. lamarck,lamarck,"[nan, nan]","[750003262.0, 750003263.0]","[rue lamarck, square lamarck]",no spaces perfect short,lamarck,lamarck
898553,bpt6k9780089g,1448,161,Bofflix,rasoirs et repasseurs de lames,. Rambuteau,14.,1922,. rambuteau,rambuteau,"[nan, nan, nan]","[750005592.0, 750005591.0, 750005593.0]","[porte rambuteau, passage rambuteau, rue rambu...",no spaces perfect short,rambuteau,rambuteau
899153,bpt6k9780089g,1475,18,Sarrault,cordonnier,. Meslay,60.,1922,. meslay,meslay,"[nan, nan]","[750006630.0, 750005711.0]","[rue meslay, passage meslay]",no spaces perfect short,meslay,meslay


In [15]:
len(non_unique_short_s)

817

In [16]:
Counter(u_short_aligned["Métier"]).most_common(50)

[('vins', 70795),
 ('épicier', 33921),
 ('propriétaire', 24982),
 ('médecin', 19539),
 ('tailleur', 17150),
 ('coiffeur', 16293),
 ('boulanger', 15343),
 ('boucher', 14592),
 ('architecte', 12883),
 ('rentier', 12258),
 ('menuisier', 11024),
 ('serrurier', 10711),
 ('chaussures', 9698),
 ('horloger', 9392),
 ('modes', 8607),
 ('peintre-artiste', 8570),
 ('limonadier', 8308),
 ('mercerie', 7983),
 ('charcutier', 7616),
 ('ébéniste', 6937),
 ('sage-femme', 6782),
 ('pharmacien', 6473),
 ('avocat', 6388),
 ('bottier', 6269),
 ('tapissier', 6017),
 ('lingerie', 5703),
 ('couturière', 5638),
 ('peintre-vitrier', 5259),
 ('vins-traiteur', 5107),
 ('bijoutier en or', 4708),
 ('tonnelier', 4543),
 ('fleuriste', 4534),
 ('libraire', 4365),
 ('vins en gros', 4349),
 ('herboriste', 4348),
 ('tabac', 4073),
 ('pâtissier', 3952),
 ('chapelier', 3900),
 ('cordonnier', 3748),
 ('propr.', 3638),
 ('mécanicien', 3589),
 ('bois et charbons', 3488),
 ('papetier', 3443),
 ('relieur', 3416),
 ('restaurateu

In [17]:
print("Aligned data so far:", (len(long_aligned)+len(u_short_aligned)+len(nu_short_aligned))/len(bottins))

Aligned data so far: 0.8189772328038629


In [18]:
long_aligned[["Nom", "Métier", "Rue", "Numéro", "source", "streetname"]].head(2)

,Nom,Métier,Rue,Numéro,source,streetname
12,Abbatucci (Séverin),député de la Corse,place Vendôme,11-13.,2022,place vendome
13,Abbatucci # (Th.),maître des requêtes au conseil d'Etat,pl. Vendôme,ll et 13.,2022,place vendome


In [19]:
long_aligned[["Nom", "Métier", "Rue", "Numéro", "source", "streetname", "no_spaces"]].tail(2)

,Nom,Métier,Rue,Numéro,source,streetname,no_spaces
1011765,Zoppett,sculpt. sur bois,r. deReuilly,18.,2022,rue de reuilly,ruedereuilly
1011773,Zurconi (P.),fourreur,quai de l'Hôtel-deVille,46.,2022,quai de l'hotel de ville,quaidelhoteldeville


In [20]:
non_unique_short_s[["streetname_short", "streetname"]].head(5)

,streetname_short,streetname
26,abbaye,"[chemin de l'abbaye, rue de l'abbaye]"
43,abbesses,"[passage des abbesses, place des abbesses, rue..."
58,acacias,"[passage des acacias, rue des acacias]"
81,adrienne,"[cite adrienne, villa adrienne]"
92,aguesseau,"[rue d'aguesseau, marche d'aguesseau]"


In [21]:
[x for x in not_aligned["Rue"] if ("5" in x)][10:20]

['52. Lauriston (Marquis de)',
 '54. Petiet',
 '5. Saint-Projet(chev. de)',
 '15',
 '15',
 '56 et q. des Augustins',
 '57. Baudon de Mony',
 '65. Chasteigner (Cte)',
 'Popincourt 5. Genie',
 '105. Lagrange (Cte C. de)']

## fuzzy matching

In [22]:
from fuzzywuzzy import process
from tqdm import tqdm

In [23]:
#ravis code
def simple_processor(token: str) -> str:
    """A string processor to return the same string as input.
        This dummy processor is used to avoid the default processor of the Rapidfuzz module to calculate string similarity.

    Parameters
    ----------
    token : str
        The input string to process.

    
    Returns
    -------
    str
        The output string same as the input string.
    """
    return token

In [45]:
not_aligned_rues = not_aligned["street"].unique().tolist()
not_aligned_selected = [street for street, _ in Counter(not_aligned["street"].tolist()).most_common(10000)]
not_aligned_selected100 = not_aligned_rues[:100]

In [25]:
streets_all_vars = streets["streetname"]
#streets_all_vars.append(streets["streetname_short"])
streets_all_vars.append(streets["street_abbr"])
streets_all_vars = list(set(streets_all_vars))
streets_all_vars

['rue albinoni',
 'voie d/8',
 'villa sommeiller',
 "rue d'hautpoul",
 'rue erasme',
 'rue du departement',
 'rue de croulebarbe',
 'rue henri chevreau',
 'passage des petits peres',
 'place chassaigne goyon',
 'allee chantal akerman',
 'cour saint nicolas',
 'rue fabert',
 'passerelle leopold sedar senghor',
 'voie ac/19',
 'rue abel rabaud',
 'rue gaston de saint paul',
 'voie eh/13',
 'rue danville',
 'villa seurat',
 'rue fessart',
 "boulevard d'aurelle de paladines",
 'rue boyer',
 "avenue de l'ecole militaire des invalides",
 'rue sorbier',
 'avenue mac mahon',
 'rond point saint charles',
 'rue des deux anges',
 "rue de la montagne d'aulas",
 'rue laurent pichat',
 'impasse massonnet',
 'rue de menil montant',
 'rue de belleville',
 'rue des patriarches',
 'square gouvion saint cyr',
 'rue geoffroy marie',
 'rue jean bologne',
 'rue darcy',
 'rue jules renard',
 'rue gustave larroumet',
 'villa des lilas',
 'place de clichy',
 'rue charles le goffic',
 'passage des fours a chaux

In [28]:
for x in not_aligned_selected100:
    best_one = process.extractOne(x, streets_all_vars, processor=simple_processor, score_cutoff=85) #90 too low
    if best_one:
        print(x, best_one)

passage choiseal ('passage choiseul', 94)
quai des angustins ("avenue de l'ecole militaire des invalides", 86)
beaujolais da temple ('voie da/12', 86)
pei.ecuries ('rue des petites ecuries', 86)
neuve desbons enfants ('rue des bons enfants', 88)
paradis poissonniere ('cite paradis', 86)
austerlitz saint marcel ('rue saint eloi', 86)
saint maar popincourt ('rue saint eloi', 86)
neuvest eustache ('souterrain boucle saint eustache', 86)
neape saint eustache ('chemin conduisant a saint denis', 86)
amandiers popincourt ('passage sainte anne popincourt', 86)
valoisdu roule ('rue saint philippe du roule', 86)
place de l'hotel de ville ("place de l'hotel de ville   esplanade de la liberation", 90)
orleans saint marcel ('chemin conduisant a saint denis', 86)
saint maur popincourt ('rue saint eloi', 86)
poitou marais ('rue des marais du temple', 86)
place saint germain l'auxerrois ("rue saint germain l'auxerrois", 90)
neave saint denis ("place saint thomas d'aquin", 86)
passerelle des pet. ecuri

In [29]:
for x in not_aligned_selected100:
    best_one = process.extractOne(x, streets_all_vars, processor=simple_processor, score_cutoff=90) #90 too low
    if best_one:
        print(x, best_one)

passage choiseal ('passage choiseul', 94)
place de l'hotel de ville ("place de l'hotel de ville   esplanade de la liberation", 90)
place saint germain l'auxerrois ("rue saint germain l'auxerrois", 90)
aux ours ('rue aux ours', 90)
henry ('rue henry de bournazel', 90)
boulevard de mont parnasse ('boulevard du mont parnasse', 96)
michodiere ('rue de la michodiere', 90)
nre sainte catherine ('rue sainte catherine', 95)
quai de gevres ('quai de gesvres', 97)
avenue lowendal ('avenue de lowendal', 95)
rochechouart ('rue marguerite de rochechouart', 90)
luxembourg ('rue neuve de luxembourg', 90)
de seze ('rue de seze', 90)
la harpe ('rue de la harpe', 90)
et boulevard de saint martin ('boulevard saint martin', 95)
quai de l'ecole ("quay de l'ecole", 93)
boulevard de capucines ('boulevard des capucines', 98)
la tour d'auvergne ("rue de la tour d'auvergne", 95)


In [48]:
fuzzy_dict = dict()
for x in tqdm(not_aligned_selected):
    best_one = process.extractOne(x, streets_all_vars, processor=simple_processor, score_cutoff=90)
    if best_one:
        fuzzy_dict[x] = best_one[0]

100%|██████████| 10000/10000 [2:07:04<00:00,  1.31it/s]    


In [49]:
# load pickle module
import pickle

f = open("fuzzy_dict10000with90.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(fuzzy_dict,f)

# close file
f.close()

In [50]:
len(fuzzy_dict)

1531

In [51]:
fuzzy_dict_clean = {key:value[0] for key, value in fuzzy_dict.items()}
not_aligned["street_fuzzy"] = not_aligned["street"].map(fuzzy_dict_clean)

In [52]:
long_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= streets, 
                    df_aligned = long_aligned, mergeOnLeft="street_fuzzy", mergeOnRight="streetname", 
                    align_method="fuzzy")
long_aligned, not_aligned = align_on_column(df_not_aligned = not_aligned, df_streets= streets, 
                    df_aligned = long_aligned, mergeOnLeft="street_fuzzy", mergeOnRight="street_abbr", 
                    align_method="abbr. fuzzy")

Joining on street_fuzzy and streetname, method:fuzzy
#total aligned: 1577782, newly aligned: 0, not aligned: 705916


MemoryError: Unable to allocate 35.4 GiB for an array with shape (8, 593371990) and data type object

In [44]:
long_aligned.tail(20)

,Unnamed: 0,page,row,Nom,Métier,Rue,Numéro,annee,street,id1791,...,street_short,street_abbr,street_long,source,streetname,streetname_short,align_method,no_spaces,no_spaces_long,street_fuzzy
886242,bpt6k9780089g,1604,261,Yrondy (J.),photographe,av,de la GrandeArmée. 87.,1922,av,NaN,...,mac mahon,av. mac mahon,avenue mac mahon,2022,avenue mac mahon,mac mahon,fuzzy,av,avenuemacmahon,avenue mac mahon
886258,bpt6k9780089g,1605,124,Zager,brocheurs,r. de la Douane,24.,1922,rue de la douane,NaN,...,durance,r. de la durance,rue de la durance,2022,rue de la durance,durance,fuzzy,ruedeladouane,ruedeladurance,rue de la durance
886261,bpt6k9780089g,1605,140,Zaigue,ingénieur E. C. L.,r. Franceur,31.,1922,rue franceur,NaN,...,francoeur,r. francoeur,rue francoeur,2022,rue francoeur,francoeur,fuzzy,ruefranceur,ruefrancoeur,rue francoeur
886266,bpt6k9780089g,1605,206,Zamkoff père,chirurgien-dentiste,boul. Rochechouart,124.,1922,boulevard de rochechouart,NaN,...,marguerite de rochechouart,bd. marguerite de rochechouart,boulevard marguerite de rochechouart,2022,boulevard marguerite de rochechouart,marguerite de rochechouart,fuzzy,boulevardderochechouart,boulevardmargueritederochechouart,boulevard marguerite de rochechouart
886267,bpt6k9780089g,1605,251,Zannettini,vins,r. de Vanves,110.,1922,rue de vanves,235.0,...,NaN,NaN,NaN,1791,rue de vannes,vannes,fuzzy,ruedevanves,ruedevannes,rue de vannes
886269,bpt6k9780089g,1605,274,Zapfle (E. Neveu e success.),extincteurs d'incendie,r. Pierre-Levée,19.,1922,rue pierre levee,NaN,...,pierre levee,r. de la pierre levee,rue de la pierre levee,2022,rue de la pierre levee,pierre levee,fuzzy,ruepierrelevee,ruedelapierrelevee,rue de la pierre levee
886270,bpt6k9780089g,1605,295,Zaslavsky,tailleur pour dames,r. de Rochechouart,74.,1922,rue de rochechouart,1293.0,...,NaN,NaN,NaN,1791,rue de rochechouard,rochechouard,fuzzy,ruederochechouart,ruederochechouard,rue de rochechouard
886271,bpt6k9780089g,1605,295,Zaslavsky,tailleur pour dames,r. de Rochechouart,74.,1922,rue de rochechouart,1402.0,...,NaN,NaN,NaN,1791,rue de rochechouard,rochechouard,fuzzy,ruederochechouart,ruederochechouard,rue de rochechouard
886272,bpt6k9780089g,1605,302,Zavadil (Jean),menus et calendriers,r. Dupuis,9.,1922,rue dupuis,121.0,...,NaN,NaN,NaN,1791,rue du puits,puits,fuzzy,ruedupuis,ruedupuits,rue du puits
886280,bpt6k9780089g,1606,93,Zemmer,épicier,av. La Bourdonnais,31.,1922,avenue la bourdonnais,NaN,...,la bourdonnais,av. de la bourdonnais,avenue de la bourdonnais,2022,avenue de la bourdonnais,la bourdonnais,fuzzy,avenuelabourdonnais,avenuedelabourdonnais,avenue de la bourdonnais
